In [2]:
import evaluate
import numpy as np
import sys
import os
from datasets import load_from_disk, disable_caching
from sklearn.metrics import f1_score
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    set_seed,
)
sys.path.append(os.path.abspath('../../modules'))
from experiment_1.BertEntity import BertEntity

In [4]:
# import random
# seeds = [random.randint(0, 1e9) for _ in range(5)]
# seeds

In [3]:
disable_caching()

In [4]:
num_labels = 5
id2label = {
    0: "reject",
    1: "B_supplies_A",
    2: "A_supplies_B",
    3: "ambiguous",
    4: "ownership",
}
label2id = {
    "reject": 0,
    "B_supplies_A": 1,
    "A_supplies_B": 2,
    "ambiguous": 3,
    "ownership": 4,
}
metric = evaluate.load("f1")

In [5]:
model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["__NE_FROM__", "__NE_TO__", "__NE_OTHER__"]}
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_micro = f1_score(labels, predictions, average='micro')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_classwise = f1_score(labels, predictions, average=None)

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        **{f"f1_class_{i}": score for i, score in enumerate(f1_classwise)}
    }


def model_init():
    model = BertEntity.from_pretrained(
        model_name,
        num_labels=5,
        id2label=id2label,
        label2id=label2id,
    )
    model.resize_token_embeddings(len(tokenizer))
    return model

In [7]:
ds = load_from_disk("../../datasets/ManualDataset")
ds = ds.select_columns(["masked_text", "label"])
ds = ds.rename_column("masked_text", "text")
ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/2559 [00:00<?, ? examples/s]

Map:   0%|          | 0/418 [00:00<?, ? examples/s]

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

In [8]:
def run_experiment(seed):
    set_seed(seed)
    training_args = TrainingArguments(
        seed=seed,
        data_seed=seed,
        tf32=True,
        output_dir="logs/experiment_1_bert_entity",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        warmup_ratio=0.1,
        load_best_model_at_end=True,
        save_total_limit=1,
        report_to=[],
        save_only_model=True,
    )
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=ds["train"],
        eval_dataset=ds["valid"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    test_results = trainer.predict(ds["test"])
    return test_results

In [9]:
seeds = [656566143, 497239539, 527721645, 74564875, 180967469]
all_results = []

for seed in seeds:
    results = run_experiment(seed)
    all_results.append(results)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.811123788356781, 'eval_f1_micro': 0.6961722488038278, 'eval_f1_macro': 0.6456507285136183, 'eval_f1_class_0': 0.7066246056782335, 'eval_f1_class_1': 0.4528301886792453, 'eval_f1_class_2': 0.7412587412587412, 'eval_f1_class_3': 0.6911764705882353, 'eval_f1_class_4': 0.6363636363636364, 'eval_runtime': 0.3852, 'eval_samples_per_second': 1085.234, 'eval_steps_per_second': 70.099, 'epoch': 1.0}


  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.8591747879981995, 'eval_f1_micro': 0.8373205741626795, 'eval_f1_macro': 0.8086969489445904, 'eval_f1_class_0': 0.8648648648648649, 'eval_f1_class_1': 0.676923076923077, 'eval_f1_class_2': 0.8603773584905661, 'eval_f1_class_3': 0.796875, 'eval_f1_class_4': 0.8444444444444444, 'eval_runtime': 0.3662, 'eval_samples_per_second': 1141.309, 'eval_steps_per_second': 73.721, 'epoch': 9.0}
{'loss': 0.0353, 'grad_norm': 12.705297470092773, 'learning_rate': 1.3888888888888892e-06, 'epoch': 9.38}


  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.8726463317871094, 'eval_f1_micro': 0.8301435406698564, 'eval_f1_macro': 0.804288968878722, 'eval_f1_class_0': 0.8502994011976048, 'eval_f1_class_1': 0.6875, 'eval_f1_class_2': 0.8614232209737828, 'eval_f1_class_3': 0.7777777777777778, 'eval_f1_class_4': 0.8444444444444444, 'eval_runtime': 0.3519, 'eval_samples_per_second': 1187.765, 'eval_steps_per_second': 76.722, 'epoch': 10.0}
{'train_runtime': 88.1156, 'train_samples_per_second': 290.414, 'train_steps_per_second': 18.158, 'train_loss': 0.2905592140555382, 'epoch': 10.0}


  0%|          | 0/47 [00:00<?, ?it/s]

In [10]:
# Calculate mean and std of F1 scores
metrics = ["test_f1_micro", "test_f1_macro"] + [f"test_f1_class_{i}" for i in range(5)]  # Adjust range based on num_classes

avg_results = {}
for metric in metrics:
    scores = [r[metric] for r in [x.metrics for x in all_results]]
    avg_results[metric] = {
        'mean': np.mean(scores),
        'std': np.std(scores)
    }

# Print results
for metric in metrics:
    print(f"Average {metric}: {avg_results[metric]['mean']:.4f} ± {avg_results[metric]['std']:.4f}")

Average test_f1_micro: 0.7895 ± 0.0252
Average test_f1_macro: 0.7756 ± 0.0305
Average test_f1_class_0: 0.8215 ± 0.0211
Average test_f1_class_1: 0.7727 ± 0.0612
Average test_f1_class_2: 0.8094 ± 0.0076
Average test_f1_class_3: 0.6664 ± 0.0324
Average test_f1_class_4: 0.8079 ± 0.0405
